In [1]:
!pip install music21

In [2]:
from music21 import converter,note,chord,stream,instrument
import glob
import pickle
import numpy as np
from keras.utils import np_utils
import os

TensorFlow 1.x selected.


Using TensorFlow backend.


## Loading notes files

In [0]:
with open('./drive/My Drive/PreprocessedData/notes','rb') as f:
    notes = pickle.load(f)
f.close()

In [20]:
n_vocab = len(set(notes))
print(len(notes))
print(n_vocab)

65730
381


## Generating the sequential data for the LSTM model

In [0]:
snotes = sorted(set(notes))

In [0]:
ele_int = dict((ele,idx) for idx,ele in enumerate(snotes))

In [24]:
ele_int["F#2"]
n = np.array(notes)
n[n=="F#4"].shape
#n.shape

(868,)

In [0]:
sequence_length = 50

network_input = []
network_output = []

for i in range(len(notes) - sequence_length):
    seq_in = notes[i:i+sequence_length] # in the string format
    seq_out = notes[i+sequence_length]
    
    network_input.append([ele_int[ch] for ch in seq_in])
    network_output.append(ele_int[seq_out])

In [26]:
print(len(network_input))
n_patterns = len(network_input)
print(network_input[0])

65680
[333, 352, 330, 333, 352, 330, 378, 378, 333, 333, 340, 66, 308, 340, 66, 308, 366, 366, 311, 311, 367, 275, 363, 367, 275, 363, 304, 275, 363, 304, 275, 363, 379, 83, 375, 379, 83, 375, 353, 333, 330, 353, 333, 330, 330, 330, 378, 378, 333, 333]


In [27]:
print(len(network_output))

65680


In [28]:
network_input = np.reshape(network_input,(n_patterns,sequence_length,1))
print(network_input.shape)

(65680, 50, 1)


In [0]:
# Normalize our data [0,1]
normalized_network_input = network_input/float(n_vocab)
#print(normalized_network_input)

In [30]:
# Converting the y values into one hot vectors
network_output = np_utils.to_categorical(network_output)
print(network_output.shape)

(65680, 381)


In [31]:
print(normalized_network_input.shape)
print(network_output.shape)

(65680, 50, 1)
(65680, 381)


## Creating the LSTM model

In [0]:
from keras.models import Sequential,load_model
from keras.layers import *
from keras.callbacks import ModelCheckpoint,EarlyStopping
from sklearn.metrics import classification_report,confusion_matrix
from keras.optimizers import Adam

In [0]:
adam = Adam(lr = 1e-3)

In [110]:
model = Sequential()
model.add(LSTM(512,input_shape = (normalized_network_input.shape[1],normalized_network_input.shape[2]),return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512, return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(512))
model.add(Dense(512,activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(n_vocab,activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy',optimizer = adam)
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 50, 512)           1052672   
_________________________________________________________________
dropout_7 (Dropout)          (None, 50, 512)           0         
_________________________________________________________________
lstm_8 (LSTM)                (None, 50, 512)           2099200   
_________________________________________________________________
dropout_8 (Dropout)          (None, 50, 512)           0         
_________________________________________________________________
lstm_9 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dense_5 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_9 (Dropout)          (None, 512)              

In [111]:
checkpoint = ModelCheckpoint("model.hdf5",monitor='loss',verbose=0,save_best_only=True,mode = 'min')

hist = model.fit(normalized_network_input,network_output,epochs = 100,batch_size = 1024,callbacks = [checkpoint])

Epoch 1/100
65680/65680 [==============================] - 57s 866us/step - loss: 4.8638
Epoch 2/100
65680/65680 [==============================] - 56s 850us/step - loss: 4.7411
Epoch 3/100
65680/65680 [==============================] - 56s 853us/step - loss: 4.7314
Epoch 4/100
65680/65680 [==============================] - 56s 853us/step - loss: 4.7259
Epoch 5/100
65680/65680 [==============================] - 56s 855us/step - loss: 4.7220
Epoch 6/100
65680/65680 [==============================] - 56s 853us/step - loss: 4.7201
Epoch 7/100
65680/65680 [==============================] - 56s 854us/step - loss: 4.6885
Epoch 8/100
65680/65680 [==============================] - 56s 852us/step - loss: 4.5849
Epoch 9/100
65680/65680 [==============================] - 56s 853us/step - loss: 4.5441
Epoch 10/100
65680/65680 [==============================] - 56s 854us/step - loss: 4.5266
Epoch 11/100
65680/65680 [==============================] - 56s 853us/step - loss: 4.5081
Epoch 12/100
65680/

In [0]:
## model.save("model.hdf5")
## model = load_model("model.hdf5")